<a href="https://colab.research.google.com/github/pctablet505/datascience-assignment/blob/main/Data_Science_Lab_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


<center>
<img src="https://upload.wikimedia.org/wikipedia/en/thumb/b/b2/Birsa_Institute_of_Technology_Sindri_logo.png/220px-Birsa_Institute_of_Technology_Sindri_logo.png">
<h1>Data Science Lab Assignment</h1>
</center>


|||
|---|---|
|Lab|Data Science|
|Professor|Tapan Nayek|
|Name|Rahul Kumar|
|Roll Number|**1809022**|
|Branch|Computer Science|
|Signature|rahul kumar|



# List of experiments
|Sl No|Experiment|Page No|Remark|
| :---- | :---: | :--- | :---: |
|1. |Introduction to Python tools for data science|||
|2.| Basic Statistics tools and Visualization tools  in Python |||
|3.| K-means Clustering |||
|4.| Linear Regression |||
|5.| Logistic Regression|||
|6.| Naive Bayesian Classifier|||
|7.| Decision Trees|||
|8.| Simulate Principal component analysis |||
|9.| Compare the efficiency, precision and recall for any three algorithm with  common data set|||



# Introductioin to Python tools for data science
**Aim:-** To demonstrate the use of different pythont tools for data science

**Algorithm Used:-** None

**Python Program:-**

```
# dummy code
print(hello world)
```

**Evaluation of model metrics :-**  None in this experiment





---

# Naive Bayesian Classifier
**Aim:-** To classify text data into different categories

**Algorithm Used:-** Naive Bayes algorithm

**Dataset Used:-** Genre Classification Dataset IMDb

[https://www.kaggle.com/hijest/genre-classification-dataset-imdb?select=Genre+Classification+Dataset](https://www.kaggle.com/hijest/genre-classification-dataset-imdb?select=Genre+Classification+Dataset)



**Python Program:-**

```
It has been implemented below
```

**Evaluation of model metrics :-**  Accuracy, Precision, Recall

## Downloading the dataset and preprocessing it



In [ ]:
#Downloading the dataset
%cd Downloads/Genre Classification/Genre Classification Dataset
%pwd



C:\Users\pctab\Downloads\Genre Classification\Genre Classification Dataset


'C:\\Users\\pctab\\Downloads\\Genre Classification\\Genre Classification Dataset'

In [28]:
# import the libraries
import pandas as pd
import numpy as np

from collections import namedtuple, defaultdict, Counter
import string
from math import log

import nltk
from nltk.corpus import stopwords
import nltk.tokenize as tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.metrics import confusion_matrix




In [ ]:
Movie=namedtuple('Movie',['id','name','genre','description'])

## Preparing the dataset

In [ ]:
# prepare the dataset
movies={}

with open('train_data.txt',encoding='utf-8') as f:
    for x in f.readlines():
        x=x.lower()
        id,movie,genre,description=x.split(':::')
        id=int(id)
        movie=movie.strip()
        genre=genre.strip()
        movies[id]=Movie(id,movie,genre,description)



In [ ]:
#download relevant nltk modules
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

#define set of stopwords
stopwords_set=set(stopwords.words('english'))




[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\pctab\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\pctab\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\pctab\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## Prepare the vocabulary

In [ ]:
# Prepare the vocabulary
vocabulary=Counter()
vocabulary_size=10000


def preprocess_text(text):
    lametizer=WordNetLemmatizer()
    stemmer=nltk.PorterStemmer()
    result=[]
    words=tokenize.word_tokenize(text)
    for word in words:
            word=lametizer.lemmatize(word)
            word=stemmer.stem(word)   
            if word not in stopwords_set and word not in string.punctuation and not word.isnumeric():
                result.append(word)
    return result


def prepare_vocabulary():    
    for x in movies:
        text=movies[x].description
        words=preprocess_text(text)
        for word in words:
            vocabulary[word]+=1
    




In [ ]:
%%time
prepare_vocabulary()
#vocabulary=Counter({word:count for word,count in vocabulary.most_common(vocabulary_size)})

Wall time: 2min 33s


In [27]:
#vocabulary

In [ ]:
print('Total number of words in vocabulary is :',len(vocabulary))


Total number of words in vocabulary is : 118065


In [ ]:
#vocabulary

In [ ]:
# the list of genres to classify the movie
genres=set([movies[x].genre for x in movies])
print(genres)
print('Number of genres in the dataset is :',len(genres))

{'fantasy', 'reality-tv', 'adventure', 'news', 'horror', 'animation', 'drama', 'short', 'game-show', 'western', 'crime', 'adult', 'musical', 'comedy', 'biography', 'romance', 'sport', 'talk-show', 'war', 'documentary', 'music', 'history', 'sci-fi', 'family', 'thriller', 'mystery', 'action'}
Number of genres in the dataset is : 27


In [ ]:
# prepare the word count table
word_count_table=defaultdict(lambda:defaultdict(int))

def prepare_word_count_table():
    for id in movies:
        description=movies[id].description
        genre=movies[id].genre
        words=preprocess_text(description)
        for word in words:
            if word in vocabulary:          
                word_count_table[word][genre]+=1
                word_count_table[word]['total']+=1


In [ ]:
%%time
prepare_word_count_table()

Wall time: 2min 33s


In [ ]:
word_count_table



## Using bayes rule to implement Naive Bayesian Classifier

**Assumption :** It assumes that each event is independent of each other, which isn't true, but still works good enough to produce usable results.


**Bayes Rule :-** 

$ P(A|B)=\frac{P(B|A) \times P(A)}{P(B)} $

$ => $ $ log(P(A|B)) = log(P(B|A)) + log(P(A)) - log(P(B)) $

**Log-Likelihood :-**

$ log(P(Genre|w_1,w_2, ... w_n)) = \sum_{i=1}^{n}log(P(w_i|Genre) + n \times log(P(Genre)) - \sum_{i=1}^{n}log(P(w_i)$




In [ ]:
# Calculate the log priors
log_priors=defaultdict(lambda:defaultdict(int))

for word in vocabulary:
    for genre in genres:
        log_priors[word][genre]=log(word_count_table[word][genre]+1)-log(word_count_table[word]['total']+len(genres))



count_genres=defaultdict(int)
for id in movies:
    count_genres[movies[id].genre]+=1

log_priors_genre={genre:np.log(count_genres[genre]+1)-np.log(len(movies)+len(genres)) for genre in genres}

total_word_counts=sum([word_count_table[x]['total'] for x in word_count_table])

log_priors_words=defaultdict(float)

for word in vocabulary:
    log_priors_words[word]=np.log(vocabulary[word]/total_word_counts)



## Make the predictions

In [ ]:
#define the prediction function
def softmax(log_posteriors):
    
    probabilities={}
    s=sum([np.exp(x) for x in log_posteriors.values()])
    for x in log_posteriors:
        probabilities[x]=np.exp(log_posteriors[x])/s
    
    
    print(sum(probabilities.values()))
    print(sorted(probabilities.items(),key=lambda x:x[1],reverse=True))
def predict(text):
    words=preprocess_text(text.lower())
    posteriors=defaultdict(float)
    for word in words:
        if word in vocabulary:
            for genre in genres:
                posteriors[genre]+=log_priors[word][genre]+log_priors_genre[genre]-log_priors_words[word]
    m=float('-inf')
    res=None
    for genre in genres:
        if posteriors[genre]>m:
            m=posteriors[genre]
            res=genre
    return res
    #softmax(posteriors)




In [ ]:
desc='''A photographer is sailing with his wife, her sister and his nympho-maniacal model. He leaves the three women alone to get a part for his boat. A mysterious man shows up, who might be an escaped criminal the police are searching for. This doesn't alarm the three women too much, and he rapidly seduces all three of them.'''
predict(desc)

'drama'

## Testing the model

In [ ]:
%%time

test_movies_predictions={}

# Load the test data and make the predictions
with open('test_data.txt',encoding='utf-8') as f:
    for x in f.readlines():
        id,name,description=x.lower().split(':::')
        y=predict(description)
        test_movies_predictions[int(id)]=y

# Load the test data solution and check the actual values
test_movies_actual={}
with open('test_data_solution.txt',encoding='utf-8') as f:
    for x in f.readlines():
        id,name,genre,description=x.lower().split(':::')
        test_movies_actual[int(id)]=genre.strip()


result=[(id,test_movies_actual[id]==test_movies_predictions[id]) for id in test_movies_actual]

# Calculate the accuracy
accuracy = sum([x[1] for x in result])/len(result)


        




Wall time: 2min 51s


In [ ]:
print('Accuracy of model : {:.3f}%'.format(accuracy*100))
truth_counts=defaultdict(int)
prediction_counts=defaultdict(int)
for id in test_movies_actual:
    truth_counts[test_movies_actual[id]]+=1
    prediction_counts[test_movies_predictions[id]]+=1


accuracy_genrewise={}
true_positives=defaultdict(int)
false_positives=defaultdict(int)
for id in test_movies_predictions:
    if test_movies_predictions[id]==test_movies_actual[id]:
        true_positives[test_movies_actual[id]]+=1
    else:
        false_positives[test_movies_predictions[id]]+=1
accuracy_genrewise={genre:true_positives[genre]/truth_counts[genre] for genre in genres}

for genre,acc in accuracy_genrewise.items():
    print('genre: {},\t\t\t accuracy: {}'.format(genre,acc))


Accuracy of model : 43.280%
genre: fantasy,			 accuracy: 0.0
genre: reality-tv,			 accuracy: 0.0
genre: adventure,			 accuracy: 0.0
genre: news,			 accuracy: 0.0
genre: horror,			 accuracy: 0.0
genre: animation,			 accuracy: 0.0
genre: drama,			 accuracy: 0.8971495739053776
genre: short,			 accuracy: 0.0
genre: game-show,			 accuracy: 0.0
genre: western,			 accuracy: 0.0
genre: crime,			 accuracy: 0.0
genre: adult,			 accuracy: 0.0
genre: musical,			 accuracy: 0.0
genre: comedy,			 accuracy: 0.00013430029546065002
genre: biography,			 accuracy: 0.0
genre: romance,			 accuracy: 0.0
genre: sport,			 accuracy: 0.0
genre: talk-show,			 accuracy: 0.0
genre: war,			 accuracy: 0.0
genre: documentary,			 accuracy: 0.8586591325595602
genre: music,			 accuracy: 0.0
genre: history,			 accuracy: 0.0
genre: sci-fi,			 accuracy: 0.0
genre: family,			 accuracy: 0.0
genre: thriller,			 accuracy: 0.0
genre: mystery,			 accuracy: 0.0
genre: action,			 accuracy: 0.0


In [29]:
confusion_matrix(test_movies_actual,test_movies_predictions)

ValueError: ignored

#Evaluation of model
After using the naive bayes classifier for **27** class classification, <br>
we got an accuracy of **43%** which is very nice for such large number of classes.

---
_






_

---

In [ ]:
import json
data={'log_prior_words':log_priors_words,
      'log_prior_genres':log_priors_genre,
      'vocabulary':vocabulary}
data_string=json.dumps(data)
with open('naive_bayes_data.json','w') as f:
    json.dump(data_string,f)

